# Re Run Step Function Executions

### Imports & Creds :

In [4]:
import boto3
import datetime
from dateutil import tz

# Set up boto3 client for Step Functions
stepfunctions = boto3.client('stepfunctions')

# Constants
STATE_MACHINE_NAME = "RAPISAMStateMachine-1YwcU3XmIcZ5"
SUPPLIER_ID = "MACKIE"
START_TIME_UTC = datetime.datetime(2025, 5, 27, 14, 0, 0, tzinfo=datetime.timezone.utc)
NOW_UTC = datetime.datetime.utcnow().replace(tzinfo=datetime.timezone.utc)

### Functons :

In [5]:
def get_state_machine_arn(name):
    print(f"[INFO] Looking up state machine ARN for '{name}'...")
    paginator = stepfunctions.get_paginator('list_state_machines')
    for page in paginator.paginate():
        for sm in page['stateMachines']:
            if sm['name'] == name:
                print(f"[INFO] Found ARN: {sm['stateMachineArn']}")
                return sm['stateMachineArn']
    raise ValueError(f"State machine '{name}' not found.")

def list_supplier_executions(state_machine_arn, supplier_id, start_time, end_time):
    print(f"[INFO] Listing executions for supplier '{supplier_id}' between {start_time} and {end_time}...")
    executions = []
    paginator = stepfunctions.get_paginator('list_executions')
    total_scanned = 0
    matched_count = 0
    for page in paginator.paginate(stateMachineArn=state_machine_arn, statusFilter='SUCCEEDED'):
        for exe in page['executions']:
            total_scanned += 1
            if supplier_id in exe['name'] and start_time <= exe['startDate'] <= end_time:
                matched_count += 1
                print(f"  [MATCH {matched_count}] Name: {exe['name']}, Start: {exe['startDate']}")
                executions.append(exe)
    print(f"[INFO] Total executions scanned: {total_scanned}")
    print(f"[INFO] Matching executions found: {matched_count}")
    return executions

def get_execution_input(execution_arn):
    print(f"[INFO] Fetching input for execution ARN: {execution_arn}")
    response = stepfunctions.describe_execution(executionArn=execution_arn)
    return response['input']

def start_retry_executions(executions, state_machine_arn):
    print(f"[INFO] Retrying {len(executions)} executions...")
    for i, exe in enumerate(executions, 1):
        new_name = f"{exe['name']}-Retry"
        input_data = get_execution_input(exe['executionArn'])
        print(f"[{i}/{len(executions)}] Starting retry: {new_name}")
        try:
            stepfunctions.start_execution(
                stateMachineArn=state_machine_arn,
                name=new_name[:80],
                input=input_data
            )
            print(f"    ✅ Success: {new_name}")
        except stepfunctions.exceptions.ExecutionAlreadyExists as e:
            print(f"    ⚠️ Skipped (already exists): {new_name}")
        except Exception as e:
            print(f"    ❌ Error: {e}")


### Re Run Executions

In [ ]:
def main():
    print("[INFO] Script started.")
    try:
        state_machine_arn = get_state_machine_arn(STATE_MACHINE_NAME)
        executions = list_supplier_executions(
            state_machine_arn, SUPPLIER_ID, START_TIME_UTC, NOW_UTC
        )

        if not executions:
            print("[INFO] No matching executions found. Exiting.")
            return

        start_retry_executions(executions, state_machine_arn)
    except Exception as e:
        print(f"[ERROR] {e}")

    print("[INFO] Script finished.")

if __name__ == "__main__":
    main()

[INFO] Script started.
[INFO] Looking up state machine ARN for 'RAPISAMStateMachine-1YwcU3XmIcZ5'...
[INFO] Found ARN: arn:aws:states:us-east-1:389633136494:stateMachine:RAPISAMStateMachine-1YwcU3XmIcZ5
[INFO] Listing executions for supplier 'MACKIE' between 2025-05-27 14:00:00+00:00 and 2025-05-28 13:17:28.101616+00:00...
  [MATCH 1] Name: MACKIE_005140_251482813091331_20250528111011.492, Start: 2025-05-28 13:10:17.641000+02:00
  [MATCH 2] Name: MACKIE_100179_251482812452479_20250528104510.1689, Start: 2025-05-28 12:45:48.304000+02:00
  [MATCH 3] Name: MACKIE_002239_251482812270921_20250528102810.8824, Start: 2025-05-28 12:28:17.023000+02:00
  [MATCH 4] Name: MACKIE_100179_251482812233775_20250528102410.786, Start: 2025-05-28 12:24:08.940000+02:00
  [MATCH 5] Name: MACKIE_008111_251482810350830_20250528083608.4542, Start: 2025-05-28 10:36:13.590000+02:00
  [MATCH 6] Name: MACKIE_004442_251482809262353_20250528072607.7895, Start: 2025-05-28 09:26:31.960000+02:00
  [MATCH 7] Name: MACKI